In [1]:
import os
import sys
import shutil
import argparse
import IPython 
from PIL import Image, ImageFont, ImageDraw
from enum import Enum
from typing import Callable, List, Optional, Tuple, Union
from functools import partial

import torch
import torch.nn as nn
import torchvision
from torchvision.datasets import VisionDataset
from torchvision.transforms import transforms
import numpy as np
import pandas as pd
import skimage
from scipy import sparse
import matplotlib.pyplot as plt 
import torchxrayvision as xrv

from dinov2.models.unet import UNet
from dinov2.data import SamplerType, make_data_loader, make_dataset
from dinov2.data.datasets import NIHChestXray, MC
from dinov2.data.loaders import make_data_loader
from dinov2.data.transforms import make_segmentation_target_transform, make_segmentation_transform
from dinov2.eval.setup import setup_and_build_model
from dinov2.eval.utils import ModelWithIntermediateLayers, ModelWithNormalize, evaluate, extract_features
from dinov2.eval.metrics import build_segmentation_metrics, MetricAveraging
from dinov2.utils import show_image_from_tensor
from dinov2.eval.segmentation import setup_decoders, TransformerEncoder, LinearDecoder
from fvcore.common.checkpoint import Checkpointer, PeriodicCheckpointer

In [2]:
args = argparse.Namespace(config_file='dinov2/configs/eval/vits14_pretrain.yaml', pretrained_weights='models/dinov2_vits14_pretrain.pth', output_dir='results/NIH/dinov2_vits14/knn', opts=[], train_dataset_str='MC:split=TRAIN:root=/mnt/z/data/MC/train', val_dataset_str='MC:split=VAL:root=/mnt/z/data/MC/test', nb_knn=[5, 20, 50, 100, 200], temperature=0.07, gather_on_cpu=False, batch_size=8, n_per_class_list=[-1], n_tries=1, ngpus=1, nodes=1, timeout=2800, partition='learnlab', use_volta32=False, comment='', exclude='')
model, autocast_dtype = setup_and_build_model(args)

I20230829 01:18:33 23211 dinov2 config.py:60] git:
  sha: eff0d2f88dd87cac50bf6981d4488304dacd7254, status: has uncommitted changes, branch: main

I20230829 01:18:33 23211 dinov2 config.py:61] batch_size: 8
comment: 
config_file: dinov2/configs/eval/vits14_pretrain.yaml
exclude: 
gather_on_cpu: False
n_per_class_list: [-1]
n_tries: 1
nb_knn: [5, 20, 50, 100, 200]
ngpus: 1
nodes: 1
opts: ['train.output_dir=/mnt/c/Users/user/Desktop/dinov2/results/NIH/dinov2_vits14/knn']
output_dir: /mnt/c/Users/user/Desktop/dinov2/results/NIH/dinov2_vits14/knn
partition: learnlab
pretrained_weights: models/dinov2_vits14_pretrain.pth
temperature: 0.07
timeout: 2800
train_dataset_str: MC:split=TRAIN:root=/mnt/z/data/MC/train
use_volta32: False
val_dataset_str: MC:split=VAL:root=/mnt/z/data/MC/test
I20230829 01:18:33 23211 dinov2 config.py:27] sqrt scaling learning rate; base: 0.004, new: 0.001
I20230829 01:18:33 23211 dinov2 config.py:34] MODEL:
  WEIGHTS: ''
compute_precision:
  grad_scaler: true
  teach

In [3]:
transform = make_segmentation_transform(resize_size=448)
target_transform = make_segmentation_target_transform(resize_size=448)

train_dataset = make_dataset(
    dataset_str=args.train_dataset_str,
    transform=transform,
    target_transform=target_transform
)
val_dataset = make_dataset(
    dataset_str=args.val_dataset_str,
    transform=transform,
    target_transform=target_transform,
)

sampler_type = SamplerType.INFINITE

I20230829 01:18:34 23211 dinov2 loaders.py:90] using dataset: "MC:split=TRAIN:root=/mnt/z/data/MC/train"
0 scans are missing from TRAIN set
I20230829 01:18:34 23211 dinov2 loaders.py:95] # of dataset samples: 92
I20230829 01:18:34 23211 dinov2 loaders.py:90] using dataset: "MC:split=VAL:root=/mnt/z/data/MC/test"
46 scans are missing from VAL set
I20230829 01:18:34 23211 dinov2 loaders.py:95] # of dataset samples: 46


In [4]:
embed_dim = model.embed_dim
decoders, optim_param_groups = setup_decoders(
    embed_dim,
    [1e-6, 2e-6, 5e-6, 1e-5, 2e-5, 5e-5, 1e-4, 2e-4, 5e-4, 1e-3, 2e-3, 5e-3, 1e-2, 5e-2, 1e-1],
    3,
)

In [5]:
output_dir = "models/trained_heads/segmentation_linear/model_final.pth"
checkpointer = Checkpointer(decoders, output_dir)
start_iter = checkpointer.resume_or_load(output_dir, resume=True).get("iteration", -1) + 1

I20230829 01:18:34 23211 fvcore.common.checkpoint checkpoint.py:150] [Checkpointer] Loading from models/trained_heads/segmentation_linear/model_final.pth ...
W20230829 01:18:34 23211 fvcore.common.checkpoint checkpoint.py:338] Skip loading parameter 'decoders_dict.segmentor_lr_0_0000010000.decoder.weight' to the model due to incompatible shapes: (3, 1024, 1, 1) in the checkpoint but (3, 384, 1, 1) in the model! You might want to double check if this is expected.
W20230829 01:18:34 23211 fvcore.common.checkpoint checkpoint.py:338] Skip loading parameter 'decoders_dict.segmentor_lr_0_0000020000.decoder.weight' to the model due to incompatible shapes: (3, 1024, 1, 1) in the checkpoint but (3, 384, 1, 1) in the model! You might want to double check if this is expected.
W20230829 01:18:34 23211 fvcore.common.checkpoint checkpoint.py:338] Skip loading parameter 'decoders_dict.segmentor_lr_0_0000050000.decoder.weight' to the model due to incompatible shapes: (3, 1024, 1, 1) in the checkpoint 

In [6]:
autocast_ctx = partial(torch.cuda.amp.autocast, enabled=True, dtype=autocast_dtype)
decoder = decoders.module.decoders_dict["segmentor_lr_0_1000000000"]
feature_model = TransformerEncoder(model, autocast_ctx=autocast_ctx)

In [57]:
highlight_multipler = 50
metric = build_segmentation_metrics(average_type=MetricAveraging.SEGMENTATION_METRICS, num_labels=3).cuda()
result_message = ""
for image, target in val_dataset:
    image, target = image.cuda(non_blocking=True).unsqueeze(0), target.cuda(non_blocking=True).unsqueeze(0)
    with torch.no_grad(): 
        features=model.forward_features(image)['x_norm_patchtokens']
    logits = decoder(features)
    logits = torch.nn.functional.interpolate(logits, size=448, mode="bilinear", align_corners=False)
    prediction = logits.argmax(dim=1)

    results = metric(prediction, target)

    target = target.squeeze()
    prediction = prediction.squeeze()

    concated = torch.cat((target, prediction), dim=-1)

    prediction = (prediction * highlight_multipler).cpu()
    target = (target * highlight_multipler).cpu()
    concated = (concated.unsqueeze(0).type(torch.int32) * highlight_multipler).cpu()
    H, W = concated.squeeze().shape

    pil_image = torchvision.transforms.ToPILImage()(concated)
    pil_image = pil_image.convert("L") # Convert to Grayscale

    draw = ImageDraw.Draw(pil_image)
    _, _, w, h = draw.textbbox((0, 0), "Target")
    draw.text(((W-w)*0.25, H*0.025), "Target", fill=255)

    _, _, w, h = draw.textbbox((0, 0), "Prediction")
    draw.text(( (W-w) * 0.75, H*0.025), "Prediction", fill=255)

    for m, r in dict(results).items():
        result_message += f"{m}: {float(r):.3f} "

    draw.text((0, 0), result_message, fill=255)

    pil_image.save("hello.png")
    print(metric.compute())
    break

{'dice': tensor(0., device='cuda:0'), 'jaccard': tensor(0., device='cuda:0')}
